### Load packages

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.tools.plotting import scatter_matrix
%matplotlib inline
from sklearn import cross_validation as cv
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import cross_validation
from sklearn import preprocessing
import numpy as np
from sklearn.linear_model import LogisticRegression

### Load csv

In [19]:
motionloc = 'C:/Users/Valued Customer/Desktop/motion_feature.csv'
nomotionloc = 'C:/Users/Valued Customer/Desktop/nomotion_feature.csv'
motion = pd.read_csv(motionloc, header=None,skip_blank_lines=True) # load whole table, row 22 is NA
nomotion = pd.read_csv(nomotionloc, header=None,skip_blank_lines=True)
motion_data = motion.ix[1:,2:]  # column 2-end are features, row 0 is column name
nomotion_data = nomotion.ix[1:,2:]
motion_class = np.empty(len(motion)-1) 
motion_class [:] = 1 #
#motion.ix[1:,7] # column 7 is class
nomotion_class = np.empty(len(nomotion)-1) 
nomotion_class [:] = 0 
#nomotion.ix[1:,7]
x=motion_data.append(nomotion_data,ignore_index = True)
x=x.astype('float64') # convert string to float
y=pd.concat([pd.DataFrame(motion_class),pd.DataFrame(nomotion_class)],ignore_index = True)
for i in range(len(y)):
    y.ix[i]=y.ix[i].astype('category') # convert string to categorical

### Exploratory analysis

In [3]:
# data_all=x
# data_all['class']=y

In [4]:
# _=data_all.boxplot(by='class')

In [5]:
# _=data_all.groupby('class').boxplot() 

In [6]:
# _=scatter_matrix(x, alpha=0.2, figsize=(6, 6))#, diagonal='kde')

In [7]:
# _=data_all.hist()

### Split training and testing

In [25]:
x_train, x_test, y_train, y_test=cv.train_test_split(x,y,test_size=0.1)

### Data preprocessing

In [26]:
x_train_scaled = preprocessing.scale(x_train)
scaler = preprocessing.StandardScaler().fit(x_train)
x_test_scaled= scaler.transform(x_test)

In [27]:
data_scaled=pd.DataFrame(x_train_scaled)
data_scaled['class']=y_train

In [16]:
# data_all=x
# data_all['class']=y
# _=data_all.boxplot(by='class')
# _=data_all.groupby('class').boxplot() 
# _=scatter_matrix(x, alpha=0.2, figsize=(6, 6))#, diagonal='kde')
# _=data_all.hist()

### Build classifier

In [28]:
names=['LDA','QDA','logistic regression','random forest','SVC','KNN']
classifiers=[LinearDiscriminantAnalysis(),
             QuadraticDiscriminantAnalysis(),
             LogisticRegression(),
             RandomForestClassifier(max_depth=5, n_estimators=3, max_features=1),
             SVC(gamma=2, C=1),
             KNeighborsClassifier(3)]
for name, clf in zip(names,classifiers):
    #clf.fit(x_train_scaled,y_train)
    #print(clf.score(x_test_scaled,y_test))
    score = cross_validation.cross_val_score(clf, x_train_scaled, y_train[0], cv=3)#,scoring=scoring)
    print'{}: accuracy {}+/-{}'.format(name,score.mean(),score.std())

LDA: accuracy 0.713981244672+/-0.0862944163702
QDA: accuracy 0.65501562944+/-0.0504466694071
logistic regression: accuracy 0.757459505541+/-0.0351000867517
random forest: accuracy 0.737993748224+/-0.0402120442201
SVC: accuracy 0.81067064507+/-0.0355238703356
KNN: accuracy 0.786516055698+/-0.0168469090048
